In [1]:
#Install the packages below

#!pip install pandas
#!pip install numpy
#!pip install pandas-datareader
#!pip install arch

In [2]:
# Import these packages
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import arch

In [4]:
Bitcoin = pdr.get_data_yahoo("BTC-USD", start="2022-04-27", end="2022-07-27")
print(Bitcoin.columns)

Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')


In [5]:
# Create new variable name stock_data
stock_data = Bitcoin[['Close']]
stock_data["log"] = np.log(stock_data)-np.log(stock_data.shift(1))
stock_data.tail()

C:\Users\TOT_User\AppData\Local\Temp\ipykernel_9260\111969909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data["log"] = np.log(stock_data)-np.log(stock_data.shift(1))


,Close,log
Date,,
2022-07-23,22465.478516,-0.011045
2022-07-24,22609.164062,0.006375
2022-07-25,21361.701172,-0.056756
2022-07-26,21239.753906,-0.005725
2022-07-27,22930.548828,0.076595


In [6]:
stock_data.describe()

,Close,log
count,93.000000,92.000000
mean,26676.690125,-0.005524
std,6142.637953,0.041463
min,19017.642578,-0.174053
25%,20860.449219,-0.025221
50%,28360.810547,-0.002464
75%,30296.953125,0.017558
max,39773.828125,0.078920


In [7]:
vol = stock_data.log.std()
vol

0.04146271734626344

## **A.**

### *Compute Historical Volatility*

In [8]:
log_returns = np.log(stock_data.Close/stock_data.Close.shift(1))
#print(log_returns)
sum_log_returns = 0
mean_log_returns = log_returns.mean()
for i in range(1,len(log_returns)):
    sum_log_returns += (log_returns.iloc[i]-mean_log_returns)**2
variance = sum_log_returns/(len(log_returns)-1)
vol = np.sqrt(variance)
print(vol)

0.04123676080112792


In [9]:
np.std(stock_data.log)

0.04123676080112789

In [10]:
st = stock_data["log"].dropna().ewm(span=365).std()
sigma = st.iloc[-1]
print(sigma)

0.04124344034168868


### *Compute Stochastic Volatility*

In [11]:
model = arch.arch_model(stock_data.log.dropna(), mean='Zero', vol='GARCH', p=1, q=1)
model_fit = model.fit()
forecast = model_fit.forecast(horizon = 1) 
var = forecast.variance.iloc[-1]
print(var)
sigma = float(np.sqrt(var))
print(sigma)

Iteration:      1,   Func. Count:      5,   Neg. LLF: -50.669070635183566
Iteration:      2,   Func. Count:     12,   Neg. LLF: -160.63353257399086
Iteration:      3,   Func. Count:     17,   Neg. LLF: -158.83522721667165
Iteration:      4,   Func. Count:     22,   Neg. LLF: -161.30373986842426
Iteration:      5,   Func. Count:     27,   Neg. LLF: -162.9106332772324
Iteration:      6,   Func. Count:     31,   Neg. LLF: -162.91075008967093
Iteration:      7,   Func. Count:     35,   Neg. LLF: -162.91075836283534
Iteration:      8,   Func. Count:     38,   Neg. LLF: -162.91075836281382
Optimization terminated successfully    (Exit mode 0)
            Current function value: -162.91075836283534
            Iterations: 8
            Function evaluations: 38
            Gradient evaluations: 8
h.1    0.00309
Name: 2022-07-27 00:00:00, dtype: float64
0.055589526534618974


C:\Users\TOT_User\anaconda3\lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0017. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\TOT_User\anaconda3\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


### *Compute Implied Volatility*

In [12]:
#!pip install scipy
import scipy.stats
from numpy import sqrt, log, exp, pi

N = scipy.stats.norm.cdf
ONE_CENT = 0.1
step = 0.0001
N_price = scipy.stats.norm.pdf
MAX_TRY = 1000

def bs_price(S, K, r, t, sigma):
    d1 = (log(S/K) + (r+sigma**2/2)*t) / (sigma*sqrt(t))
    d2 = d1 - sigma * sqrt(t)
    Put_Price = N(-d2) * K * exp(-r*t) - N(-d1) * S 
    return Put_Price,d1,d2 


def brute_force(S, K, r, t, market_price):
    _sigma = 0.5
    for i in range(10000): #max number of calculations is 10000
        Put_Price = bs_price(S, K, r, t, sigma = _sigma)
        diff = market_price - Put_Price[0]
        if diff > ONE_CENT:
            _sigma = _sigma + step
        elif diff < 0 and abs(diff) > ONE_CENT:
            _sigma = _sigma - step
        elif abs(diff) < ONE_CENT:
            return _sigma
    return _sigma
    
'''def find_iv_newton(S, K, r, t, market_price):
    _sigma = 0.5
    for i in range(MAX_TRY):
        Put_Price = bs_price(S, K, r, t, sigma=_sigma)
        diff = market_price - Put_Price[0]
        vega = S*N_price(Put_Price[1])*sqrt(t)
        if abs(diff) < ONE_CENT:
            return _sigma
        _sigma += diff/vega
    return _sigma'''

'def find_iv_newton(S, K, r, t, market_price):\n    _sigma = 0.5\n    for i in range(MAX_TRY):\n        Put_Price = bs_price(S, K, r, t, sigma=_sigma)\n        diff = market_price - Put_Price[0]\n        vega = S*N_price(Put_Price[1])*sqrt(t)\n        if abs(diff) < ONE_CENT:\n            return _sigma\n        _sigma += diff/vega\n    return _sigma'

In [13]:
#!pip install times
import time
start = time.time()
sigma = brute_force(21500,17200,0.02,30/365, 21239.75)
print(sigma)
print(f'finished in {time.time() - start} seconds')
'''
start = time.time()
sigma2 = find_iv_newton(21500,17200,0.02,30/365, 21239.75)
print(sigma2)
print(f'finished in {time.time() - start} seconds')'''

1.4999999999998899
finished in 3.936225175857544 seconds


"\nstart = time.time()\nsigma2 = find_iv_newton(21500,17200,0.02,30/365, 21239.75)\nprint(sigma2)\nprint(f'finished in {time.time() - start} seconds')"

## B.

### **Actual Volatility**
         This is the measure of the amount of randomness in an asset return at any particular time. It is very difficult to measure, but is supposed to be an input into all option pricing models.
    • There is no ‘timescale’ associated with actual volatility; it is a quantity that exists at each instant, possibly 
    varying from moment to moment.

### **Historical Volatility**
        This is a measure of the amount of randomness over some period in the past. The period is always specified, and so 
        is the mathematical method for its calculation. Sometimes this backward-looking measure is used as an estimate for 
        what volatility will be in the future.
        • There are two ‘timescales’ associated with historical volatility: one short, and one long.
Example: The 60-day volatility using daily returns. Perhaps of interest if you are pricing a 60-day option, which you are hedging daily.In pricing an option we are making an estimate of what actual volatility will be over the lifetime of the option. After the option has expired we can go back and calculate what the volatility actually was over the life of the option.
    
      Example: I sold a 30-day option for a 30% volatility, I hedged it every day. Did I make money?

### Implied Volatility
    The implied volatility is the volatility which when input into the Black–Scholes option pricing formulae gives the       market price of the option. It is often described as the market’s view of the future actual volatility over the lifetime of the particular option. However, it is also influenced by other effects such as supply and demand. 
    • There is one ‘timescale’ associated with implied volatility: expiration.

### Stochastic Volatility 
    refers to the fact that the volatility of asset prices varies and is not constant, as is assumed in the Black Scholes options pricing model. Stochastic volatility modeling attempts to correct for this problem with Black Scholes by allowing volatility to fluctuate over time.

## C. Pricing Put Option

### i) Black Scholes Model

In [14]:
from scipy.stats import norm 
import matplotlib.pyplot as plt

In [15]:
def calculate_put_option_price(S,T,r,sigma): 
    """
    Calculates price for put option according to the formula.        
    Formula: PresentValue(K)*N(-d2) - S*N(-d1)
    """
    T /= 365
    K = 0.8*S
    # cumulative function of standard normal distribution (risk-adjusted probability that the option will be exercised)    
    d1 = (np.log(S/K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    # cumulative function of standard normal distribution (probability of receiving the stock at expiration of the option)
    d2 = (np.log(S/K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    return (K * np.exp(-r * T) * norm.cdf(-d2, 0.0, 1.0) - S * norm.cdf(-d1, 0.0, 1.0))

In [16]:
S_0 = 21500
T = 30
r = 0.02
sigma = 1.5
Put_Price = calculate_put_option_price(S_0,T,r,sigma)
Put_Price

1558.5089728080457

### ii) Finite Difference Method

In [17]:
#!pip install mplot3d-dragger
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse

In [18]:
def bottom_boundary_condition(K,T, r, t):
    return K*np.exp(-r*(T-t))
def top_boundary_condition(t):
    return np.zeros(t.shape)
def final_boundary_condition(K,S):
    return np.maximum(K-S,0)

In [19]:
def compute_abc( K, T, sigma, r, S, dt, dS ):
    a = -sigma**2 * S**2/(2* dS**2 ) + r*S/(2*dS)
    b = r + sigma**2 * S**2/(dS**2)
    c = -sigma**2 * S**2/(2* dS**2 ) - r*S/(2*dS)
    return a,b,c
def compute_lambda( a,b,c ):
    return scipy.sparse.diags( [a[1:],b,c[:-1]],offsets=[-1,0,1])
def compute_W(a,b,c, V0, VM): 
    M = len(b)+1
    W = np.zeros(M-1)
    W[0] = a[0]*V0 
    W[-1] = c[-1]*VM 
    return W

In [20]:
def price_put_explicit( K, T, r, sigma, N, M):
    # Choose the shape of the grid
    dt = T/N
    S_min=0
    S_max=K*np.exp(8*sigma*np.sqrt(T))
    dS = (S_max-S_min)/M
    S = np.linspace(S_min,S_max,M+1)
    t = np.linspace(0,T,N+1)
    V = np.zeros((N+1,M+1)) #...
    
    # Set the boundary conditions
    V[:,-1] = top_boundary_condition(t)
    V[:,0] = bottom_boundary_condition(K,T, r, t)
    V[-1,:] = final_boundary_condition(K,S) #...
    
    # Apply the recurrence relation
    a,b,c = compute_abc(K,T,sigma,r,S[1:-1],dt,dS)
    Lambda =compute_lambda( a,b,c) 
    identity = scipy.sparse.identity(M-1)
    
    for i in range(N,0,-1):
        W = compute_W(a,b,c,V[i,0],V[i,M])
        # Use `dot` to multiply a vector by a sparse matrix
        V[i-1,1:M] = (identity-Lambda*dt).dot( V[i,1:M] ) - W*dt
        
    return V, t, S

In [21]:
K = 17200  #0.8*21500
T = 30/365
r = 0.02
sigma =1.5
N = 50
M = 50
price_put_explicit( K, T, r, sigma, N, M)[0]

array([[ 1.71717493e+04,  6.88983565e+03,  1.80945347e+03, ...,
        -6.04771567e+02,  8.40442940e+01,  0.00000000e+00],
       [ 1.71723138e+04,  6.88071260e+03,  1.78144699e+03, ...,
         1.62178259e+01, -1.52870550e+00,  0.00000000e+00],
       [ 1.71728784e+04,  6.87160723e+03,  1.75320258e+03, ...,
        -3.17352030e-01,  1.52089025e-02,  0.00000000e+00],
       ...,
       [ 1.71988691e+04,  6.48337815e+03,  9.46217233e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.71994345e+04,  6.47592981e+03,  4.76368224e+01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.72000000e+04,  6.46854173e+03,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

### iii) Monte Carlo Simulation

In [22]:
def MonteCarloPricing(S_0,T,r,sigma,N):
    num_of_steps = T
    T /= 365
    K = 0.8*S_0
    dt = T/num_of_steps
    
    np.random.seed(20)
    simulation_results = None

    # Initializing price movements for simulation: rows as time index and columns as different random price movements.
    S = np.zeros((num_of_steps,N))        
    # Starting value for all price movements is the current spot price
    S[0] = S_0

    for t in range(1,num_of_steps):
        # Random values to simulate Brownian motion (Gaussian distibution)
        Z = np.random.standard_normal(N)
        # Updating prices for next point in time 
        S[t] = S[t - 1] * np.exp((r - 0.5*sigma ** 2) * dt + (sigma * np.sqrt(dt) * Z))

    simulation_results_S = S
    
    if simulation_results_S is None:
            return -1
    return np.exp(-r*T) * 1/N * np.sum(np.maximum(K - simulation_results_S[-1], 0))
    
    

In [23]:
S_0 = 21500
T = 30
r = 0.02
sigma = 1.5
N = 10000
print(MonteCarloPricing(S_0,T,r,sigma,N))

1455.7826560912142


In [ ]:
# 1)I decide to use implied volatility to calculated.Because,it's the highest value.
# 2) -

###  iv) 


In [ ]:
''' Black–Scholes model
    - The Black–Scholes formula gives a theoretical estimate of the price of European-style options and shows that the option has a unique price given the risk of the security and its expected to return.
'''
# Source: https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model
''' Finite difference methods
    - Finite difference methods are used to price options by approximating the differential equation that describes how an option price evolves over time by
      a set of (discrete-time) difference equations.
    - This method can solve derivative pricing problems that have, in general, the same level of complexity as those problems solved by tree approaches,but,
      given their relative complexity, are usually employed only when other approaches are inappropriate.
      At the same time, like tree-based methods, this approach is limited in terms of the number of underlying variables, and for problems with Monte Carlo methods for option pricing are usually preferred.
'''
# Source: https://en.wikipedia.org/wiki/Finite_difference_methods_for_option_pricing
''' Monte Carlo methods
    - Monte Carlo option model uses Monte Carlo methods[Notes 1] to calculate the value of an option with multiple sources of uncertainty or with complicated features. 
    - Monte Carlo Methods are particularly useful in the valuation of options with multiple sources of uncertainty or with complicated features, which would make them difficult to value 
      through a straightforward Black–Scholes-style or lattice based computation.
    - The technique is thus widely used in valuing path dependent structures like lookback- and Asian options and in real options analysis.Additionally, as above, 
      the modeller is not limited as to the probability distribution assumed.
    - If an analytical technique for valuing the option exists—or even a numeric technique, such as a (modified) pricing tree—Monte Carlo methods will usually be too slow to be competitive.  
'''    
# Source: https://en.wikipedia.org/wiki/Monte_Carlo_methods_for_option_pricing
'''From these 3 models,In my opinion I think the 1st model;Black Scholes model; is the best model.Because,it gives the highest put pricing value.
# The value of 2nd models,I think the value is over.'''
